In [30]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import root_mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.ensemble import IsolationForest
from scipy.stats import zscore
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from patsy import dmatrices

In [31]:
import kagglehub
from kagglehub import KaggleDatasetAdapter

file_path = "austinHousingData.csv"


df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "ericpierce/austinhousingprices",
  file_path,
  # Provide any additional arguments like 
    pandas_kwargs={"sep": ","}
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
)

print("First 5 records:", df.head())

/tmp/ipykernel_10491/4082401683.py:7: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  df = kagglehub.load_dataset(


First 5 records:          zpid          city          streetAddress  zipcode  \
0   111373431  pflugerville   14424 Lake Victor Dr    78660   
1   120900430  pflugerville     1104 Strickling Dr    78660   
2  2084491383  pflugerville    1408 Fort Dessau Rd    78660   
3   120901374  pflugerville     1025 Strickling Dr    78660   
4    60134862  pflugerville  15005 Donna Jane Loop    78660   

                                         description   latitude  longitude  \
0  14424 Lake Victor Dr, Pflugerville, TX 78660 i...  30.430632 -97.663078   
1  Absolutely GORGEOUS 4 Bedroom home with 2 full...  30.432673 -97.661697   
2  Under construction - estimated completion in A...  30.409748 -97.639771   
3  Absolutely darling one story home in charming ...  30.432112 -97.661659   
4  Brimming with appeal & warm livability! Sleek ...  30.437368 -97.656860   

   propertyTaxRate  garageSpaces  hasAssociation  ...  numOfMiddleSchools  \
0             1.98             2            True  ...     

In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15171 entries, 0 to 15170
Data columns (total 47 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   zpid                        15171 non-null  int64  
 1   city                        15171 non-null  object 
 2   streetAddress               15171 non-null  object 
 3   zipcode                     15171 non-null  int64  
 4   description                 15169 non-null  object 
 5   latitude                    15171 non-null  float64
 6   longitude                   15171 non-null  float64
 7   propertyTaxRate             15171 non-null  float64
 8   garageSpaces                15171 non-null  int64  
 9   hasAssociation              15171 non-null  bool   
 10  hasCooling                  15171 non-null  bool   
 11  hasGarage                   15171 non-null  bool   
 12  hasHeating                  15171 non-null  bool   
 13  hasSpa                      151

In [34]:
usable_columns = df.select_dtypes(include=['int64', 'float64'])
usable_columns = usable_columns.drop(columns=['zpid'])

In [35]:
usable_columns.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15171 entries, 0 to 15170
Data columns (total 33 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   zipcode                     15171 non-null  int64  
 1   latitude                    15171 non-null  float64
 2   longitude                   15171 non-null  float64
 3   propertyTaxRate             15171 non-null  float64
 4   garageSpaces                15171 non-null  int64  
 5   parkingSpaces               15171 non-null  int64  
 6   yearBuilt                   15171 non-null  int64  
 7   latestPrice                 15171 non-null  float64
 8   numPriceChanges             15171 non-null  int64  
 9   latest_salemonth            15171 non-null  int64  
 10  latest_saleyear             15171 non-null  int64  
 11  numOfPhotos                 15171 non-null  int64  
 12  numOfAccessibilityFeatures  15171 non-null  int64  
 13  numOfAppliances             151

In [36]:

X = usable_columns.drop("latestPrice", axis=1)
y = usable_columns["latestPrice"]

iso = IsolationForest(contamination=.01, random_state=42)
outliers = iso.fit_predict(X) == 1
X, y = X[outliers], y[outliers]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"RMSE: {rmse:.4f}")


RMSE: 222073.4426


In [37]:
X_var = sm.add_constant(X)
vardata = pd.DataFrame()
vardata['feature'] = X_var.columns
vardata['VIF'] = [variance_inflation_factor(X_var.values, i) for i in range(X_var.shape[1])]
print(vardata)

                       feature           VIF
0                        const  3.252572e+07
1                      zipcode  1.172143e+00
2                     latitude  2.562504e+00
3                    longitude  3.622607e+00
4              propertyTaxRate  1.811404e+00
5                 garageSpaces  3.820160e+02
6                parkingSpaces  3.802563e+02
7                    yearBuilt  1.644599e+00
8              numPriceChanges  1.062408e+00
9             latest_salemonth  1.142459e+00
10             latest_saleyear  1.718523e+00
11                 numOfPhotos  1.257458e+00
12  numOfAccessibilityFeatures  1.021816e+00
13             numOfAppliances  1.100672e+00
14        numOfParkingFeatures  2.034951e+00
15  numOfPatioAndPorchFeatures  1.845628e+00
16       numOfSecurityFeatures  1.558429e+00
17     numOfWaterfrontFeatures  1.010378e+00
18         numOfWindowFeatures  1.260718e+00
19      numOfCommunityFeatures  1.585551e+00
20                 lotSizeSqFt  1.002479e+00
21        

In [38]:
model = sm.OLS(y, X).fit()
print(model.summary())

                                 OLS Regression Results                                
Dep. Variable:            latestPrice   R-squared (uncentered):                   0.795
Model:                            OLS   Adj. R-squared (uncentered):              0.795
Method:                 Least Squares   F-statistic:                              1817.
Date:                Tue, 17 Jun 2025   Prob (F-statistic):                        0.00
Time:                        22:50:04   Log-Likelihood:                     -2.1038e+05
No. Observations:               15019   AIC:                                  4.208e+05
Df Residuals:                   14987   BIC:                                  4.211e+05
Df Model:                          32                                                  
Covariance Type:            nonrobust                                                  
                                 coef    std err          t      P>|t|      [0.025      0.975]
-------------------------